In [1]:
import pandas as pd

In [22]:
bank_df = pd.read_excel("bank_deposit.xlsx")
bank_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
0,95,retired,divorced,primary,no,2282.0,no,no,telephone,21.0,apr,207.0,17.0,-1.0,0.0,unknown,yes
1,95,retired,married,secondary,no,0.0,no,no,telephone,1.0,oct,215.0,1.0,-1.0,0.0,unknown,no
2,94,retired,divorced,secondary,no,1234.0,no,no,cellular,3.0,mar,212.0,1.0,-1.0,0.0,unknown,no
3,93,retired,married,unknown,no,775.0,no,no,cellular,22.0,jul,860.0,2.0,177.0,7.0,success,yes
4,93,retired,married,unknown,no,775.0,no,no,cellular,4.0,aug,476.0,2.0,13.0,9.0,success,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11217,50,entrepreneur,divorced,tertiary,no,315.0,no,no,cellular,12.0,jul,101.0,2.0,185.0,2.0,failure,no
11218,50,management,married,tertiary,no,2881.0,no,no,cellular,5.0,aug,510.0,2.0,2.0,5.0,other,no
11219,50,technician,divorced,secondary,no,0.0,yes,no,cellular,13.0,aug,120.0,2.0,91.0,6.0,failure,no
11220,50,services,married,secondary,no,2376.0,yes,no,cellular,24.0,sep,395.0,1.0,-1.0,0.0,unknown,yes


In [23]:
# This line is to check NAN values in the DataFrame.
bank_df.isna().sum()
# This line is to check NAN values in the "contact" column.
bank_df[bank_df["contact"].isna()]


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
134,80,housemaid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,failure,no
135,80,retired,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,no
136,80,retired,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,success,yes
137,80,retired,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,no
138,80,retired,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,failure,yes
139,80,retired,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,success,yes


In [18]:
# Dropped the rows that contained NAN in the contact column.
bank_df= bank_df[bank_df['contact'].notna()]
bank_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
0,95,retired,divorced,primary,no,2282.0,no,no,telephone,21.0,apr,207.0,17.0,-1.0,0.0,unknown,yes
1,95,retired,married,secondary,no,0.0,no,no,telephone,1.0,oct,215.0,1.0,-1.0,0.0,unknown,no
2,94,retired,divorced,secondary,no,1234.0,no,no,cellular,3.0,mar,212.0,1.0,-1.0,0.0,unknown,no
3,93,retired,married,unknown,no,775.0,no,no,cellular,22.0,jul,860.0,2.0,177.0,7.0,success,yes
4,93,retired,married,unknown,no,775.0,no,no,cellular,4.0,aug,476.0,2.0,13.0,9.0,success,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11217,50,entrepreneur,divorced,tertiary,no,315.0,no,no,cellular,12.0,jul,101.0,2.0,185.0,2.0,failure,no
11218,50,management,married,tertiary,no,2881.0,no,no,cellular,5.0,aug,510.0,2.0,2.0,5.0,other,no
11219,50,technician,divorced,secondary,no,0.0,yes,no,cellular,13.0,aug,120.0,2.0,91.0,6.0,failure,no
11220,50,services,married,secondary,no,2376.0,yes,no,cellular,24.0,sep,395.0,1.0,-1.0,0.0,unknown,yes
